## AWS Bedrock LLM Deployment

To use Amazon Bedrock, start by signing up for an AWS account(https://aws.amazon.com/aispl/registration-confirmation/). Once you have successfully logged in, go to Amazon Bedrock Console and get started. Keep in mind that, by default, users do not have model access. 


You should request access from Model Access Page (https://us-east-1.console.aws.amazon.com/bedrock/home?region=us-east-1#/modelaccess). For this application, we’re going to use “Titan Text G1 Express” model by Amazon. Unfortunately, there is no free tier for Amazon Bedrock, therefore, you might need to provide payment information at this step.

##### Import all needed packages

In [20]:
import os
from os.path import join, dirname
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())
AWS_KEY=os.getenv("AWS_KEY")
AWS_SECRET_KEY=os.getenv("AWS_SECRET_KEY")

In [3]:
import boto3
import json
import base64
import os

### Step 1: Setting Up the Python Environment


 ### Setup the Bedrock runtime

In [4]:
session = boto3.Session(profile_name='default')

### Set up the boto3 object for bedrock



In [21]:
bedrock = boto3.client(service_name='bedrock', 
region_name='us-east-1', 
aws_access_key_id=AWS_KEY, 
aws_secret_access_key=AWS_SECRET_KEY)

### Write command to get all the modelId for the claude models that are available through your account



In [22]:
response = bedrock.list_foundation_models(byProvider="anthropic")

for summary in response["modelSummaries"]:
    print(summary["modelId"])

anthropic.claude-instant-v1:2:100k
anthropic.claude-instant-v1
anthropic.claude-v2:0:18k
anthropic.claude-v2:0:100k
anthropic.claude-v2:1:18k
anthropic.claude-v2:1:200k
anthropic.claude-v2:1
anthropic.claude-v2
anthropic.claude-3-sonnet-20240229-v1:0:28k
anthropic.claude-3-sonnet-20240229-v1:0:200k
anthropic.claude-3-sonnet-20240229-v1:0
anthropic.claude-3-haiku-20240307-v1:0:48k
anthropic.claude-3-haiku-20240307-v1:0:200k
anthropic.claude-3-haiku-20240307-v1:0


* Install Boto3: Boto3 is the Amazon Web Services (AWS) SDK for Python. It allows Python developers to write software that makes use of all AWS services like Amazon Bedrock, Amazon S3 and Amazon EC2. Install it using pip.

* AWS Credentials: Ensure you have your AWS access key (AWS_KEY) and AWS secret access key (AWS_SECRET_KEY). These are necessary for authentication with AWS services.

* Import these libraries in your Python file:

## Step 2: Set up parameters to access Claude

In [23]:
# modelId obtained from printing out modelIds in previous step
modelId = 'anthropic.claude-v2:1'

### parameters for the LLM to control text-generation
# temperature increases randomness as it increases
temperature = 0.5
# top_p increases more word choice as it increases
top_p = 1
# maximum number of tokens to generate in the output
max_tokens_to_generate = 250

Once we have set up the parameters it is time to create our prompts.

Claude takes both a system prompt, and a list of messages. The format should be like below. Note: the order of the messages is important and must start with the user role, and alternate with the assistant role

In [24]:
system_prompt = "All your responses should be in Haiku form"

messages = [{"role": "user", "content": "Hello, world, tell me a funny poem"}, 
            {"role": "assistant", "content": "Here is a haiku poem for you:\n\nLaughing out loud\nAt silly jokes and stories\nBringing joyful smiles"},
            {"role": "user", "content": "I don't want to smile"}]


### Step 3: Prepare the variables to call the Bedrock service
Create a new boto3 object for bedrock-runtime. Note: this is different from the boto3 object above that uses bedrock NOT bedrock-runtime

In [25]:
bedrock_runtime = boto3.client(service_name='bedrock-runtime', 
region_name='us-east-1', 
aws_access_key_id=AWS_KEY, 
aws_secret_access_key=AWS_SECRET_KEY)

2. Prepare your JSON body to send to bedrock. We have created all the variables before, and additionally we pass the anthropic version. Note: there are other parameters available through Bedrock as well, give the docs a read for more info

In [26]:
body = json.dumps({
            "messages": messages,
            "system": system_prompt,
            "max_tokens": max_tokens_to_generate,
            "temperature": temperature,
            "top_p": top_p,
            "anthropic_version": "bedrock-2023-05-31"
})

## 3. Send data to Bedrock



In [27]:
response = bedrock_runtime.invoke_model(body=body, modelId=modelId, accept="application/json", contentType="application/json")


### 4. Read the generated data!



In [28]:
response_body = json.loads(response.get('body').read())
result = response_body.get('content', '')
print(result)

[{'type': 'text', 'text': 'I understand, my friend\nSmiles can be tiring'}]


### Final Python Script
##### The full script looks as follows. Remember to execute the printing modelID part first before fully running it all



In [30]:
import boto3
import json

AWS_KEY=os.getenv("AWS_KEY")
AWS_SECRET_KEY=os.getenv("AWS_SECRET_KEY")

bedrock = boto3.client(service_name='bedrock', 
region_name='us-east-1', 
aws_access_key_id=AWS_KEY, 
aws_secret_access_key=AWS_SECRET_KEY)

response = bedrock.list_foundation_models(byProvider="anthropic")

for summary in response["modelSummaries"]:
    print(summary["modelId"])

# modelId obtained from printing out modelIds in previous step
modelId = 'anthropic.claude-v2:1'

### parameters for the LLM to control text-generation
# temperature increases randomness as it increases
temperature = 0.5
# top_p increases more word choice as it increases
top_p = 1
# maximum number of tokens togenerate in the output
max_tokens_to_generate = 250

system_prompt = "All your responses should be in Haiku form"

messages = [{"role": "user", "content": "Hello, world, tell me a funny poem"}, 
            {"role": "assistant", "content": "Here is a haiku poem for you:\n\nLaughing out loud\nAt silly jokes and stories\nBringing joyful smiles"},
            {"role": "user", "content": "I don't want to smile"}]

bedrock_runtime = boto3.client(service_name='bedrock-runtime', 
region_name='us-east-1', 
aws_access_key_id=AWS_KEY, 
aws_secret_access_key=AWS_SECRET_KEY)

body = json.dumps({
            "messages": messages,
            "system": system_prompt,
            "max_tokens": max_tokens_to_generate,
            "temperature": temperature,
            "top_p": top_p,
            "anthropic_version": "bedrock-2023-05-31"
})

response = bedrock_runtime.invoke_model(body=body, modelId=modelId, accept="application/json", contentType="application/json")

response_body = json.loads(response.get('body').read())
result = response_body.get('content', '')
print(result)

anthropic.claude-instant-v1:2:100k
anthropic.claude-instant-v1
anthropic.claude-v2:0:18k
anthropic.claude-v2:0:100k
anthropic.claude-v2:1:18k
anthropic.claude-v2:1:200k
anthropic.claude-v2:1
anthropic.claude-v2
anthropic.claude-3-sonnet-20240229-v1:0:28k
anthropic.claude-3-sonnet-20240229-v1:0:200k
anthropic.claude-3-sonnet-20240229-v1:0
anthropic.claude-3-haiku-20240307-v1:0:48k
anthropic.claude-3-haiku-20240307-v1:0:200k
anthropic.claude-3-haiku-20240307-v1:0
[{'type': 'text', 'text': 'I understand, friend\nNot always in mood for mirth\nAnother time, maybe'}]
